In [1]:
from copy import deepcopy
import drawsvg as draw
import math

In [2]:
class Drawing:
    height = 100
    width = 200
    margin = 3
    height_with_margin = height + margin * 2
    width_with_margin = width + margin * 2


class Circle:
    diameter = 35
    radius = diameter / 2
    circumference = math.pi * diameter


class Ray:
    count = 12
    spacing = 1.75
    width = Circle.circumference / 2 / count
    height = Drawing.height - Circle.diameter / 2
    height_with_spacing = height - (spacing*3)
    width_with_spacing = width - (spacing / 2)
    angle_increment = 180 / count
    center = height / 2
    half_height = (height_with_spacing / 2)
    half_width = (width - spacing / 2)

    bottom = dict(
        x=Circle.radius + spacing,
        y=half_width / -2,
        width=half_height,
        height=half_width,
    )

    top = dict(
        x=Circle.radius + half_height + spacing*2,
        y=half_width / -2,
        width=half_height,
        height=half_width,
    )


class TraceStyle:
    fill = "none"
    stroke_width = 0.635
    stroke = "black"
    style = {"fill": fill, "stroke_width": stroke_width, "stroke": stroke}


class VccTrace:
    spacing_interval = 1.25
    via_diam = 0.635


class SolderPad:
    w = 3
    h = 1
    x_spacing = 10
    y_spacing = 1
    y_offset = 4.2
    right_start_x = x_spacing / 2 + w / 2
    left_start_x = -right_start_x


class BottomGroundTrace:
    # non-parametric
    trace_v = 6.75
    trace_h = 13.5


class Resistor:
    count = 2
    pad_h = 3.35
    pad_w = 1.225
    pad_spacing = 4.7
    width = (pad_w * 2) + pad_spacing
    spacing = (Ray.half_height - (width * count)) / (count + 1)


class Diode:
    pad_h_w = 1.1
    pad_distance = 2.28
    width = pad_h_w * 2 + pad_distance
    x_offset = 3
    diode_y = -pad_h_w / 2
    diode_x1 = Ray.bottom["x"] + x_offset
    diode_x2 = (
       Ray.top["x"] + Ray.half_height - width - Ray.half_height + width + x_offset
    )

class TopGroundArc:
    radius = Ray.top["x"] + Ray.half_height - TraceStyle.stroke_width / 2


class Via:
    radius = 0.2
    copper = 1

In [3]:
sin = lambda a: math.sin(math.radians(a))
cos = lambda a: math.cos(math.radians(a))
tan = lambda a: math.tan(math.radians(a))

def line_angle(length, angle):
    x = cos(angle) * length
    y = -sin(angle) * length

    return x, y

In [4]:
def rotate(obj):
    group = draw.Group()
    for i in range(Ray.count):
        angle = -Ray.angle_increment / 2 - i * Ray.angle_increment
        _obj = deepcopy(obj)
        _obj.args['transform'] = f'rotate({angle})'
        group.append(_obj)
    return group

In [5]:
def get_drawing():
    width = Drawing.width_with_margin
    height = Drawing.height_with_margin
    d = draw.Drawing(width, height, origin=(-width / 2, -height), displayInline=False)
    d.svg_args['width'] = f'{width}mm'
    d.svg_args['height'] = f'{height}mm'
    d.svg_args['viewBox'] = f'0 0 {width} {height}'

    d.set_pixel_scale(96 / 25.4)

    return d

In [6]:
edge_layer = draw.Group(
    id="Edge.Cuts", inkscape__groupmode="layer", inkscape__label="Edge.Cuts"
)

edge_layer.args[
    "transform"
] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height_with_margin})"

back_copper_layer = draw.Group(
    id="B.Cu", inkscape__groupmode="layer", inkscape__label="B.Cu"
)
back_copper_layer.args[
    "transform"
] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"

front_copper_layer = draw.Group(
    id="F.Cu", inkscape__groupmode="layer", inkscape__label="F.Cu"
)
front_copper_layer.args[
    "transform"
] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"

back_solder_mask = draw.Group(
    id="B.Mask", inkscape__groupmode="layer", inkscape__label="B.Mask"
)
back_solder_mask.args[
    "transform"
] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"

front_solder_mask = draw.Group(
    id="F.Mask", inkscape__groupmode="layer", inkscape__label="F.Mask"
)
front_solder_mask.args[
    "transform"
] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"

drill_layer = draw.Group(
    id="Drill",
    inkscape__groupmode="layer",
    inkscape__label="Drill",
    fill="black",
    stroke_width=0,
)
drill_layer.args[
    "transform"
] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"


def edge():
    p = draw.Path(
        fill='none', stroke_width=0.1, stroke='black'
    )
    p.M(-Drawing.width_with_margin/2, -Drawing.height_with_margin)
    p.h(Drawing.width_with_margin)
    p.v(Drawing.height_with_margin)
    p.h(-Drawing.width_with_margin)
    p.Z()

    return p

edge_layer.append(edge())

In [7]:
def vcc_traces():
    center = Circle.radius + Ray.center
    traces = draw.Group(**{**TraceStyle.style, 'stroke': 'black'})
    first = draw.Path()

    for i in (3, 7, 11):
        first.M(cos(-15 * i - 7.5) *  center, sin(-15 * i - 7.5) *  center)
        first.l(
            *line_angle(
                VccTrace.spacing_interval * 3 + TraceStyle.stroke_width / 2,
                -180 + 15 * i + 7.5,
            )
        )

    first_arc_radius =  center - VccTrace.spacing_interval * 3
    first_arc_start_deg = -15 * 3 - 7.5
    first_arc_end_deg = -15 * 11 - 7.5
    first.M(
        first_arc_radius * cos(first_arc_start_deg),
        first_arc_radius * sin(first_arc_start_deg),
    )
    first.A(
        first_arc_radius,
        first_arc_radius,
        0,
        0,
        0,
        first_arc_radius * cos(first_arc_end_deg),
        first_arc_radius * sin(first_arc_end_deg),
    )

    second = draw.Path()
    for i, spacing in [
        (2, -1),
        (3.5, -1),
        (3.5, 2),
        (6, -1),
        (6, 2),
        (7.5, 1),
        (7.5, -1),
        (10, 1),
    ]:
        second.M(cos(-15 * i - 7.5) *  center, sin(-15 * i - 7.5) *  center)
        s = VccTrace.spacing_interval * abs(spacing) + TraceStyle.stroke_width / 2
        if spacing < 0:
            s = -s
        second.l(*line_angle(s, -180 + 15 * i + 7.5))

    for start_deg, end_deg, spacing in [
        (-15 * 2 - 7.5, -15 * 4, 1),
        (-15 * 4, -15 * 6 - 7.5, -2),
        (-15 * 6 - 7.5, -15 * 8, 1),
        (-15 * 8, -15 * 10 - 7.5, -1),
    ]:
        radius =  center + VccTrace.spacing_interval * spacing
        second.M(radius * cos(start_deg), radius * sin(start_deg))
        second.A(radius, radius, 0, 0, 0, radius * cos(end_deg), radius * sin(end_deg))

    third = draw.Path()
    for i, spacing in [
        (1, 3),
        (5, 3),
        (9, 3),
    ]:
        third.M(cos(-15 * i - 7.5) *  center, sin(-15 * i - 7.5) *  center)
        s = VccTrace.spacing_interval * abs(spacing) + TraceStyle.stroke_width / 2
        if spacing > 0:
            s = -s
        third.l(*line_angle(s, -180 + 15 * i + 7.5))

    radius =  center + VccTrace.spacing_interval * 3
    third.M(radius * cos(-15 - 7.5), radius * sin(-15 - 7.5))
    third.A(
        radius,
        radius,
        0,
        0,
        0,
        radius * cos(-15 * 9 - 7.5),
        radius * sin(-15 * 9 - 7.5),
    )

    forth = draw.Path()
    for i, spacing in [
        (0, -1),
        (1.5, -1),
        (1.5, 2),
        (4, 2),
        (4, -1),
        (5.5, -1),
        (5.5, 2),
        (8, 2),
    ]:
        forth.M(cos(-15 * i - 7.5) *  center, sin(-15 * i - 7.5) *  center)
        s = VccTrace.spacing_interval * abs(spacing) + TraceStyle.stroke_width / 2
        if spacing > 0:
            s = -s
        forth.l(*line_angle(s, -180 + 15 * i + 7.5))

    for start_deg, end_deg, spacing in [
        (-7.5, -15 * 2, -1),
        (-15 * 2, -15 * 4.5, 2),
        (-15 * 4.5, -15 * 6, -1),
        (-15 * 4.5, -15 * 6, -1),
        (-15 * 6, -15 * 8 - 7.5, 2),
    ]:
        radius =  center + VccTrace.spacing_interval * spacing
        forth.M(radius * cos(start_deg), radius * sin(start_deg))
        forth.A(radius, radius, 0, 0, 0, radius * cos(end_deg), radius * sin(end_deg))

    traces.append(first)
    traces.append(second)
    traces.append(third)
    traces.append(forth)

    for i in range(12):
        c = draw.Circle(
            cos(-15 * i - 7.5) *  center,
            sin(-15 * i - 7.5) *  center,
            r=VccTrace.via_diam / 2,
            fill='black',
            stroke_width=0,
        )
        traces.append(c)

    return traces

front_copper_layer.append(vcc_traces())

In [8]:
def vcc_via_pads():
    pads = draw.Group()
    params = {'width': SolderPad.w , 'height': SolderPad.h, 'fill': 'black', 'stroke_width': 0}

    for i in (1, 4):
        pads.append(
            draw.Rectangle(
                x=SolderPad. x_spacing / 2,
                y=-(SolderPad.h + SolderPad.y_offset + (SolderPad.y_spacing + SolderPad.h) * i),
                **params,
            )
        )

    for i in (1,):
        pads.append(
            draw.Rectangle(
                x=-SolderPad. x_spacing / 2 - SolderPad.w ,
                y=-(SolderPad.h + SolderPad.y_offset + (SolderPad.y_spacing + SolderPad.h) * i),
                **params,
            )
        )
    return pads


front_copper_layer.append(vcc_via_pads())

In [9]:
def vcc_con_traces():
    traces = draw.Group(**{**TraceStyle.style, 'stroke': 'black'})

    # vcc trace connections
    vcc_con1 = draw.Path()
    vcc_con1.M(
        SolderPad.left_start_x,
        -(
            SolderPad.y_offset
            + 2 * SolderPad.h
            + 2 * SolderPad.y_spacing
            + TraceStyle.stroke_width / 2
        )
        - (SolderPad.h - TraceStyle.stroke_width) / 2,
    )
    vcc_con1.h(-SolderPad.w / 2)
    vcc_con1.L(-Circle.radius * sin(60), -Circle.radius * cos(60))
    vcc_con1.l(
        *line_angle(
            Ray.center
            - VccTrace.spacing_interval * 3
            + TraceStyle.stroke_width / 2,
            90 + 60,
        )
    )

    vcc_con2 = draw.Path()
    vcc_con2.M(
        SolderPad.left_start_x,
        -(
            SolderPad.y_offset
            + 3 * SolderPad.h
            + 3 * SolderPad.y_spacing
            + TraceStyle.stroke_width / 2
        )
        - (SolderPad.h - TraceStyle.stroke_width) / 2,
    )
    vcc_con2.h(-SolderPad.w / 2)
    vcc_con2.L(-Circle.radius * sin(45), -Circle.radius * cos(45))
    vcc_con2.l(
        *line_angle(
            Ray.center
            - VccTrace.spacing_interval * 1
            + TraceStyle.stroke_width / 2,
            90 + 45,
        )
    )

    vcc_con3 = draw.Path()
    vcc_con3.M(
        SolderPad.right_start_x,
        -(
            SolderPad.y_offset
            + 3 * SolderPad.h
            + 3 * SolderPad.y_spacing
            + TraceStyle.stroke_width / 2
        )
        - (SolderPad.h - TraceStyle.stroke_width) / 2,
    )
    vcc_con3.h(SolderPad.w / 2)
    vcc_con3.L(Circle.radius * sin(45), -Circle.radius * cos(45))
    vcc_con3.l(
        *line_angle(
            Ray.center
            + VccTrace.spacing_interval * 3
            + TraceStyle.stroke_width / 2,
            45,
        )
    )

    vcc_con4 = draw.Path()
    vcc_con4.M(
        SolderPad.right_start_x,
        -(
            SolderPad.y_offset
            + 2 * SolderPad.h
            + 2 * SolderPad.y_spacing
            + TraceStyle.stroke_width / 2
        )
        - (SolderPad.h - TraceStyle.stroke_width) / 2,
    )
    vcc_con4.h(SolderPad.w / 2)
    vcc_con4.L(Circle.radius * sin(60), -Circle.radius * cos(60))
    vcc_con4.l(
        *line_angle(
            Ray.center
            - VccTrace.spacing_interval * 1
            + TraceStyle.stroke_width / 2,
            90 - 60,
        )
    )

    middle_top_gnd = draw.Path()
    middle_top_gnd.M(
        SolderPad.left_start_x,
        -(
            SolderPad.y_offset
            + 4 * SolderPad.h
            + 4 * SolderPad.y_spacing
            # + SolderPad.h / 2
        ),
    )
    middle_top_gnd.v(-SolderPad.h / 2)
    middle_top_gnd.L(-Circle.radius * sin(30), -Circle.radius * cos(30))
    middle_top_gnd.l(
        *line_angle(
            TopGroundArc.radius - Circle.radius + TraceStyle.stroke_width / 2,
            90 + 30,
        )
    )

    traces.extend([vcc_con1, vcc_con2, vcc_con3, vcc_con4, middle_top_gnd])
    return traces

back_copper_layer.append(vcc_con_traces())

In [10]:
def bottom_ground_traces():
    traces = draw.Group(
        **{**TraceStyle.style, 'stroke': 'black'}
    )
    for start_deg, end_deg in [
        (-7.5, -7.5 + -15 * 3),
        (-7.5 + -15 * 4, -7.5 + -15 * 7),
        (-7.5 + -15 * 8, -7.5 + -15 * 11),
    ]:
        traces.append(
            draw.Arc(
                0,
                0,
                Ray.bottom['x'] + Diode.pad_h_w/2,
                start_deg,
                end_deg,
            )
        )

    traces.extend(
        [
            draw.Path()
            .M(
                SolderPad. x_spacing / 2 + TraceStyle.stroke_width / 2,
                -(SolderPad.y_offset + (SolderPad.h + SolderPad.y_spacing) * 4),
            )
            .v(-BottomGroundTrace.trace_v),
            draw.Path()
            .M(
                SolderPad. x_spacing / 2,
                -(
                    SolderPad.y_offset
                    + SolderPad.h
                    + SolderPad.y_spacing
                    + TraceStyle.stroke_width / 2
                    + (SolderPad.h - TraceStyle.stroke_width) / 2
                ),
            )
            .h(BottomGroundTrace.trace_h),
            draw.Path()
            .M(
                -SolderPad. x_spacing / 2,
                -(
                    SolderPad.y_offset
                    + SolderPad.h
                    + SolderPad.y_spacing
                    + TraceStyle.stroke_width / 2
                    + (SolderPad.h - TraceStyle.stroke_width) / 2
                ),
            )
            .h(-BottomGroundTrace.trace_h),
        ]
    )

    return traces

front_copper_layer.append(bottom_ground_traces())

In [11]:
def connection_pads():
    pads = draw.Group()
    params = {'width': SolderPad.w , 'height': SolderPad.h, 'fill': 'black', 'stroke_width': 0}

    for i in range(5):
        pads.append(
            draw.Rectangle(
                x=SolderPad. x_spacing / 2,
                y=-(SolderPad.h + SolderPad.y_offset + (SolderPad.y_spacing + SolderPad.h) * i),
                **params,
            )
        )

    for i in range(5):
        pads.append(
            draw.Rectangle(
                x=-SolderPad. x_spacing / 2 - SolderPad.w ,
                y=-(SolderPad.h + SolderPad.y_offset + (SolderPad.y_spacing + SolderPad.h) * i),
                **params,
            )
        )
    return pads

back_copper_layer.append(connection_pads())
back_solder_mask.append(connection_pads())

In [12]:
def top_ground_traces():
    traces = draw.Group(**{**TraceStyle.style, 'stroke':'black'})
    right_trace = draw.Path()

    start_y = -(SolderPad.y_offset + SolderPad.h / 2)

    right_trace.M(SolderPad.right_start_x, start_y)
    right_trace.V(0)
    right_trace.H(Drawing.width - (Drawing.width - TopGroundArc.radius) + TraceStyle.stroke_width / 2)
    traces.append(
        draw.Arc(0, 0, TopGroundArc.radius, 0, -(7.5 + 15 * 3), stroke_linecap='square')
    )

    left_trace = draw.Path()
    left_trace.M(SolderPad.left_start_x, start_y)
    left_trace.V(0)
    left_trace.H(-Drawing.width + (Drawing.width - TopGroundArc.radius) - TraceStyle.stroke_width / 2)
    traces.append(
        draw.Arc(0, 0, TopGroundArc.radius, -15 * 8.5, -180, stroke_linecap='square')
    )

    traces.append(
        draw.Arc(
            0, 0, TopGroundArc.radius, -(7.5 + 15 * 4), -15 * 8, stroke_linecap='square'
        )
    )

    traces.append(right_trace)
    traces.append(left_trace)

    return traces

back_copper_layer.append(top_ground_traces())

In [13]:
def resistor_traces():
    g = draw.Group()
    p = draw.Path(
        fill='none', stroke_width=Ray.width_with_spacing, stroke='black'
    )
    
    p.M(Ray.bottom['x'], 0)
    p.h(Diode.x_offset + Diode.pad_h_w)
    p.m(Diode.pad_distance, 0)
    p.h(Diode.pad_h_w + Resistor.spacing - Diode.x_offset - Diode.width)
        
    for _ in range(Resistor.count):
        p.h(Resistor.pad_w)
        p.m(Resistor.pad_spacing, 0)
        p.h(Resistor.pad_w + Resistor.spacing)

    p.m(Ray.spacing, 0)
    p.h(Diode.x_offset + Diode.pad_h_w)
    p.m(Diode.pad_distance, 0)
    p.h(Diode.pad_h_w + Resistor.spacing - Diode.x_offset - Diode.width)
    
    for _ in range(Resistor.count):
        p.h(Resistor.pad_w)
        p.m(Resistor.pad_spacing, 0)
        p.h(Resistor.pad_w + Resistor.spacing)

    middle = draw.Path(
        fill='none', stroke_width=TraceStyle.stroke_width, stroke='black'
    )
    
    middle.M(Circle.radius + Ray.half_height + Ray.spacing, 0)
    middle.h(Ray.spacing)

    g.append(p)
    g.append(middle)
    
    return g
    
back_copper_layer.append(rotate(resistor_traces()))

In [14]:
def resistors():
    resistor_group = draw.Group(stroke_width=0, fill="black")
    y = -Resistor.pad_h + Ray.half_width / 2 - (Ray.half_width - Resistor.pad_h) / 2

    for x_offset in (
        Ray.bottom["x"],
        Ray.top['x']
    ):
        for r in range(Resistor.count):
            g = draw.Group()
            g.append(
                draw.Rectangle(
                    x_offset + Resistor.spacing + r * (Resistor.spacing + Resistor.width),
                    y,
                    Resistor.pad_w,
                    Resistor.pad_h,
                )
            )
            g.append(
                draw.Rectangle(
                    x_offset
                    + Resistor.spacing
                    + +Resistor.pad_w
                    + Resistor.pad_spacing
                    + r * (Resistor.spacing + Resistor.width),
                    y,
                    Resistor.pad_w,
                    Resistor.pad_h,
                )
            )
            resistor_group.append(g)

    return resistor_group


back_copper_layer.append(rotate(resistors()))
back_solder_mask.append(rotate(resistors()))

In [15]:
def diodes():
    diode_group = draw.Group(stroke_width=0, fill="black")
    y = -Diode.pad_h_w + Ray.half_width / 2 - (Ray.half_width - Diode.pad_h_w) / 2
    
    diode_group.append(draw.Rectangle(Diode.diode_x1, y, Diode.pad_h_w, Diode.pad_h_w))
    diode_group.append(
        draw.Rectangle(
            Diode.diode_x1 + Diode.pad_distance + Diode.pad_h_w,
            y,
            Diode.pad_h_w,
            Diode.pad_h_w,
        )
    )
    
    diode_group.append(draw.Rectangle(Diode.diode_x2, y, Diode.pad_h_w, Diode.pad_h_w))
    diode_group.append(
        draw.Rectangle(
            Diode.diode_x2 + Diode.pad_distance + Diode.pad_h_w,
            y,
            Diode.pad_h_w,
            Diode.pad_h_w,
        )
    )

    return diode_group


back_copper_layer.append(rotate(diodes()))
back_solder_mask.append(rotate(diodes()))

In [16]:
def front_heat_spreader():
    heat_group = draw.Group(stroke_width=0, fill="black")

    heat_group.extend(
        [
            draw.Rectangle(
                x=Ray.bottom["x"] + TraceStyle.stroke_width + 1,
                y=Ray.bottom["y"],
                width=Ray.bottom["width"] - VccTrace.spacing_interval * 4 - 1,
                height=Ray.bottom["height"],
            ),
            draw.Rectangle(
                x=Ray.top["x"] + VccTrace.spacing_interval * 4 - 1,
                y=Ray.top["y"],
                width=Ray.top["width"] - VccTrace.spacing_interval * 4 + 1,
                height=Ray.bottom["height"],
            ),
        ]
    )
    return heat_group


front_copper_layer.append(rotate(front_heat_spreader()))

In [17]:
def heating_trace_vias():
    via_group = draw.Group()
    # x = Ray.bottom['x'] + HeatingTrace.width / 2 - Via.copper / 2
    x = Ray.bottom["x"] + Diode.pad_h_w/2

    y = Ray.bottom['y'] + Ray.width_with_spacing / 2
    params = {'cy': y, 'r': Via.radius, 'fill': 'black', 'stroke': 0}
    via_group.extend(
        [
            draw.Circle(x, **params),
            # draw.Circle(
            #     Diode.diode_x2 + Diode.pad_h_w/2 + Diode.pad_distance,
            #     **params,
            # ),
        ]
    )

    via_group.append(
        draw.Circle(Circle.radius + Ray.center, y, r=Via.radius, fill='black', stroke_width=0)
    )

    return via_group

def connection_vias():
    via_group = draw.Group()
    v = [
        (-60, Circle.radius + Ray.center - VccTrace.spacing_interval * 3),
        (-45, Circle.radius + Ray.center - VccTrace.spacing_interval * 1),
        (45,  Circle.radius + Ray.center + VccTrace.spacing_interval * 3),
        (60,  Circle.radius + Ray.center - VccTrace.spacing_interval),
    ]
    for a, r in v:
        via_group.append(
            draw.Circle(
                r * sin(a), -r * cos(a), r=Via.radius, fill='black', stroke_width=0
            )
        )

    return via_group


def pad_vias():
    via_group = draw.Group()
    params = dict(
        r=Via.radius,
        fill='black',
        stroke_width=0,
    )

    for i in (1, 4):
        via_group.append(
            draw.Circle(
                SolderPad. x_spacing / 2 + (SolderPad.w  - Via.radius) / 2 + Via.radius / 2,
                -(
                    SolderPad.y_offset
                    + (SolderPad.h - Via.radius) / 2
                    + Via.radius / 2
                    + (SolderPad.y_spacing + SolderPad.h) * i
                ),
                **params
            )
        )
    for i in (1,):
        via_group.append(
            draw.Circle(
                -(SolderPad. x_spacing / 2 + (SolderPad.w  - Via.radius) / 2 + Via.radius / 2),
                -(
                    SolderPad.y_offset
                    + (SolderPad.h - Via.radius) / 2
                    + Via.radius / 2
                    + (SolderPad.y_spacing + SolderPad.h) * i
                ),
                **params
            )
        )

    return via_group


drill_layer.append(rotate(heating_trace_vias()))
drill_layer.append(pad_vias())
drill_layer.append(connection_vias())

In [21]:
drawing = get_drawing()

drawing.append(edge_layer)
drawing.append(front_copper_layer)
drawing.append(back_copper_layer)
drawing.append(drill_layer)
drawing.append(back_solder_mask)
drawing.append(front_solder_mask)
drawing.save_svg('outputs/sunshine_pcb.svg')

In [22]:
def clock_face_laser_cut():
    drawing = get_drawing()
    style = {"stroke_width": 0.1, "stroke": "black", "fill": "none"}
    clock_face_group = draw.Group()
    clock_face_group.args[
        "transform"
    ] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"

    outline_group = draw.Group(**style)

    ray_group = draw.Group()
    ray_group.extend(
        [
            draw.Rectangle(**Ray.bottom),
            draw.Rectangle(**Ray.top),
        ]
    )
    outline_group.append(rotate(ray_group))

    circle_radius = Circle.radius + Ray.spacing / 2
    outline_group.append(
        draw.Path(**style)
        .M(circle_radius, 0)
        .A(circle_radius, circle_radius, 0, 0, 0, -circle_radius, 0)
        .Z()
    )

    edge_group = edge()
    edge_group.args["transform"] = f"translate(0, {Drawing.margin * 1.5})"

    clock_face_group.append(outline_group)
    clock_face_group.append(edge_group)
    drawing.append(clock_face_group)

    return drawing

clock_face_laser_cut().save_svg("outputs/mask_laser_cut.svg")

In [23]:
def clock_face_silkscreen():
    drawing = get_drawing()
    style = {"stroke_width": 0, "fill": "black"}
    clock_face_group = draw.Group()
    clock_face_group.args[
        "transform"
    ] = f"translate({Drawing.width_with_margin / 2}, {Drawing.height + Drawing.margin / 2})"

    outline_group = draw.Group(**style)

    ray = {
        'x': Circle.radius + Ray.spacing - Ray.spacing/4,
        'y': (Ray.half_width + 2) / -2,
        'width': Ray.half_height*2 + Ray.spacing +2 + Ray.spacing/4,
        'height': Ray.half_width + 2,
    }

    ray_group = draw.Group()
    ray_group.append(draw.Rectangle(**ray))
    outline_group.append(rotate(ray_group))

    clock_face_group.append(outline_group)

    edge_group = edge()
    edge_group.args["transform"] = f"translate(0, {Drawing.margin * 1.5})"
    clock_face_group.append(edge_group)

    drawing.append(clock_face_group)

    return drawing

clock_face_silkscreen().save_svg("outputs/silkscreen.svg")